In [1]:
import os
import sys
from requests import Session
from time import sleep
import logging
from google.cloud import pubsub_v1
from concurrent import futures

In [11]:
class PublishTopic:
    def __init__(self):
        self.project_id = "sixth-tribute-318822"
        self.topic_id = "chidvilas-crypto"
        self.publisher = pubsub_v1.PublisherClient()
        self.topic_path = self.publisher.topic_path(self.project_id,self.topic_id)
        self.publish_futures = []

    def getcoc_loc_data(self):
        url="https://api.clashofclans.com/v1/locations"
        sess = Session()
        raw_data = sess.get(url)
        if 200<=raw_data.status_code<400:
            logging.info("Data ingested")
            return raw_data.text
        else:
            raise Exception("Failed to fetch data")


    def get_callback(self, publish_future, data):
        def callback(publish_future):
            try:
                # Wait 60 seconds for the publish call to succeed.
                logging.info(publish_future.result(timeout=60))
            except futures.TimeoutError:
                logging.error("Publishing data timed out.")

        return callback

    def pushToTopic(self,data):
        # When you publish a message, the client returns a future.
        publish_future = self.publisher.publish(self.topic_path, data.encode("utf-8"))
        # Non-blocking. Publish failures are handled in the callback function.
        publish_future.add_done_callback(self.get_callback(publish_future, data))
        self.publish_futures.append(publish_future)

        # Wait for all the publish futures to resolve before exiting.
        futures.wait(self.publish_futures, return_when=futures.ALL_COMPLETED)
        logging.info("Published message to Topic")

In [12]:
if __name__ == "__main__":
    
    
    serv = PublishTopic()
    for i in range(60):
        message=serv.getcoc_loc_data()
        serv.pushToTopic(message)
        

INFO:root:Data ingested
INFO:root:2577676002278506
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676039202301
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676112564486
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676779822144
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676028701351
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577677005378929
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577634747637184
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676095432741
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676077884928
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676765102121
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:root:2577676115872384
INFO:root:Published message to Topic
INFO:root:Data ingested
INFO:roo

In [8]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= r"D:\EGEN\keys\sixth-tribute-318822-59d11718f008.json"